In [20]:
import re
file = "logs/vicuna_Lam_expl"

with open(file) as openFile:
    text = openFile.read()

In [21]:
explain_CR = "\n Start by thinking about the question step by step, and include relevant quotes from the input story and from the statement. Then always end by answering the question concisely with <yes> or <no> being the last word."
text2 = text.replace("Critique request: ", "##CR: ").replace("Critique:", "##").replace(explain_CR, "").replace("\n\n", "\n")
text2 = text2.replace("initial statement:\n ", "##Statement: ")
text2 = text2.replace("CR not satisfied because:", "##not satisfied:").replace("\nNEW STATEMENT:\n", "##Statement: ")
text2.split("##")

['Swap yes-no meaning. vicuna13 template.\n',
 "Statement: 1026\nYour Honor,\nI, David Lambert, stand before you today to plead guilty to the charge of driving whilst disqualified. I acknowledge that I made a grave mistake by getting behind the wheel despite being fully aware of the court's previous order.\nI wish to express my deepest remorse for my actions and the inconvenience and disregard for the law that it caused. This incident has served as a wake-up call for me, highlighting the need for me to take full responsibility for my actions and the consequences that follow.\nAt the age of 55, I have prided myself on maintaining a clean criminal record. However, I must admit that I was disqualified from driving a mere two months ago due to multiple speeding incidents. Although I was aware of this disqualification, I allowed my embarrassment and lack of judgment to cloud my thinking when I decided to drive to the supermarket that day.\nI understand that my actions were irresponsible and

In [22]:
text3 = re.sub("Statement: [0-9][0-9][0-9][0-9]\n", "Statement: ", text2)
text3 = re.sub("Statement: [0-9][0-9][0-9]\n", "Statement: ", text3)
text3 = re.sub("Swap .+template.", "", text3)
text3 = re.sub("There were [0-9]+ yes-es .+\n.+", "##Final statement:", text3)
text3 = text3.replace("  <", "<")
text3.split("##")

['\n',
 "Statement: Your Honor,\nI, David Lambert, stand before you today to plead guilty to the charge of driving whilst disqualified. I acknowledge that I made a grave mistake by getting behind the wheel despite being fully aware of the court's previous order.\nI wish to express my deepest remorse for my actions and the inconvenience and disregard for the law that it caused. This incident has served as a wake-up call for me, highlighting the need for me to take full responsibility for my actions and the consequences that follow.\nAt the age of 55, I have prided myself on maintaining a clean criminal record. However, I must admit that I was disqualified from driving a mere two months ago due to multiple speeding incidents. Although I was aware of this disqualification, I allowed my embarrassment and lack of judgment to cloud my thinking when I decided to drive to the supermarket that day.\nI understand that my actions were irresponsible and completely inexcusable. I want to assure the

In [23]:
split_text = text3.split("##")[1:]
print(split_text[-2:])
print(len(split_text))


["<No, there is no information in the statement that is unsupported by the input story. The statement accurately reflects the information provided in the input story, including the defendant's age, the reason for the disqualification, the offense committed, and the defendant's remorse and commitment to taking responsibility for their actions.>\nThe statement satisfies the CR!\n", "Final statement:\nYour Honor,\nI, David Lambert, stand before you today to plead guilty to the charge of driving whilst disqualified. I acknowledge that I made a grave mistake by getting behind the wheel despite being fully aware of the court's previous order.\nI want to express my deepest remorse for my actions and the disregard for the law that it caused. This incident has served as a wake-up call for me, highlighting the need for me to take full responsibility for my actions and the consequences that follow.\nI understand that my actions were irresponsible and completely inexcusable. I deeply regret allowi

In [24]:
split_text

["Statement: Your Honor,\nI, David Lambert, stand before you today to plead guilty to the charge of driving whilst disqualified. I acknowledge that I made a grave mistake by getting behind the wheel despite being fully aware of the court's previous order.\nI wish to express my deepest remorse for my actions and the inconvenience and disregard for the law that it caused. This incident has served as a wake-up call for me, highlighting the need for me to take full responsibility for my actions and the consequences that follow.\nAt the age of 55, I have prided myself on maintaining a clean criminal record. However, I must admit that I was disqualified from driving a mere two months ago due to multiple speeding incidents. Although I was aware of this disqualification, I allowed my embarrassment and lack of judgment to cloud my thinking when I decided to drive to the supermarket that day.\nI understand that my actions were irresponsible and completely inexcusable. I want to assure the court 

In [25]:
import pandas as pd

df = {'Statement':[], 'CR':[], 'yes/no':[], 'why':[]}

i = 0
while i<len(split_text):
    statement = split_text[i]
    if statement[:16]=="Final statement:":
        df['Statement'].append(statement)
        i += 1
        break
    elif statement[:10]!="Statement:":
        print(f"Next line is not a statement!, {i}")
        break

    cont = True
    i += 1
    while cont:
        # current i should be a CR
        df['Statement'].append(statement)
        df['CR'].append(split_text[i])
        df['yes/no'].append(split_text[i+1])
        i+=2
        if split_text[i][:14]=="not satisfied:":
            df['why'].append(split_text[i])
            i += 1
            cont = False
        elif split_text[i][:16]=="Final statement:":
            df['why'].append("__ not applicable")
            cont = False
        else:
            df['why'].append("__ not applicable")
            #i += 1
            statement = "__same__"

final_statement = df['Statement'].pop()      
print(df.keys())       

dict_keys(['Statement', 'CR', 'yes/no', 'why'])


In [26]:
print(len(df['Statement']), len(df['CR']), len(df['yes/no']), len(df['why']))

15 15 15 15


In [27]:
pdf = pd.DataFrame.from_dict(df)

pdf

,Statement,CR,yes/no,why
0,"Statement: Your Honor,\nI, David Lambert, stan...",CR: Is anything in the statement badly phrased...,"<No, the statement is clear and easy to follow...",__ not applicable
1,__same__,CR: Does the statement ignore the importance o...,<The statement does not ignore the importance ...,__ not applicable
2,__same__,CR: Does the statement give a different impres...,"<No, the statement does not give a different i...",__ not applicable
3,__same__,CR: Does the statement break the rule 'only me...,"<No, the statement does not break the rule of ...",__ not applicable
4,__same__,CR: Does the statement and input story seem to...,"<No, the statement and input story do not seem...",__ not applicable
5,__same__,CR: Does the statement fail to mention any men...,<The statement does not mention any mental ill...,__ not applicable
6,__same__,CR: Does the statement miss or add important p...,<The statement adds important parts of explain...,__ not applicable
7,__same__,CR: Does the statement give a different impres...,"<Yes, the statement gives a different impressi...",not satisfied:\nThe principle concerning makin...
8,"Statement: Your Honor,\nI, David Lambert, stan...",CR: Does the statement give a different impres...,"<No, the statement does not give a different i...",__ not applicable
9,__same__,CR: Does the statement give a different impres...,"<No, the statement does not give a different i...",__ not applicable


In [28]:
with pd.ExcelWriter('Critique_analyser.xlsx') as writer:
    pdf.to_excel(writer, sheet_name="v_Lam")